In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [64]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [65]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [66]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
# X.columns

In [67]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X[col_names] = features

In [68]:

features = X_test[col_names]
scaler_test = StandardScaler().fit(features.values)
features = scaler_test.transform(features.values)
X_test[col_names] = features

### 10 Features

In [69]:
def model_accuracy(selected_features,X,X_test,y):
    X = X[selected_features]
    X_test = X_test[selected_features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    sm = SMOTE(sampling_strategy='minority', random_state = 42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    kfold = StratifiedKFold(n_splits=3, shuffle = False)
    def evaluate(model, X_train, y_train, X_test, y_test):
        print("TRAIN")
        train_predictions = model.predict_proba(X_train)
        train_pred = model.predict(X_train)

        train_accuracy = accuracy_score(y_train, train_pred)
        train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])

        print("Classification report")
        print(classification_report(y_train, train_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(train_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
        print("*" * 100)

        print("TEST")
        test_predictions = model.predict_proba(X_test)
        test_pred = model.predict(X_test)

        test_accuracy = accuracy_score(y_test, test_pred)
        test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])

        print("Classification report")
        print(classification_report(y_test, test_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(test_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
        print("*" * 100)
    def create_model(learn_rate=0.01, momentum=0):
        # create model
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    print("BASED MODEL")
    ann_accuracy = evaluate(ann, X_train, y_train, X_test, y_test)

    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = 2, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    rand_best_params = ann_random.best_params_
    print(ann_random.best_params_)
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)


    print("GRID SEARCH ANN EXPERIMENT")
    #     from random search
    #     'momentum': 0.0, 'learn_rate': 0.3, 'epochs': 100, 'batch_size': 10
    # learn_rate = [0.3, 0.1, 0.2]
    # momentum = [0.0, 0.2, 0.4]
    # batch_size = [5, 10, 20]
    # epochs = [50, 100, 150]
    learn_rate = [rand_best_params['learn_rate'],rand_best_params['learn_rate']*1.5,rand_best_params['learn_rate']/1.5]
    momentum = [rand_best_params['momentum'],rand_best_params['momentum']*1.5,rand_best_params['momentum']/1.5]
    batch_size = [rand_best_params['batch_size'],rand_best_params['batch_size']*1.5,rand_best_params['batch_size']/1.5]
    epochs = [rand_best_params['epochs'],rand_best_params['epochs']*1.5,rand_best_params['epochs']/1.5]
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    # ann_grid = rand_best_params
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=2, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    ann_best_grid = ann_grid.best_estimator_
    grid_best_parms = ann_grid.best_params_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)


# Other number of Features

In [60]:
selected_features_15 = ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry']
selected_features_20 =['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy']
selected_features_25= ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction']
selected_features_30=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction']
selected_features_35=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction']
selected_features_40=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction', 'mean_url_per_auction', 'total_no_of_participated_auctions', 'std_country_per_auction', 'ip_entropy', 'mean_country_per_auction']



In [ ]:
model_accuracy(selected_features_15,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.6495    0.8980    0.7538      1333
         1.0     0.8348    0.5154    0.6373      1333

    accuracy                         0.7067      2666
   macro avg     0.7421    0.7067    0.6955      2666
weighted avg     0.7421    0.7067    0.6955      2666

FBeta Score
0.5580828594638506
Model Performance
Accuracy = 0.7067%.
AUC ROC = 0.6242%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9705    0.9116    0.9401       577
         1.0     0.1774    0.4074    0.2472        27

    accuracy                         0.8891       604
   macro avg     0.5739    0.6595    0.5937       604
weighted avg     0.9350    0.8891    0.9091       604

FBeta Score
0.32352941176470584
Model Performance
Accuracy = 0.8891%.
AUC ROC = 0.5259%.
*

2021-11-05 23:55:09.309906: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 23:55:09.309907: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 23:55:09.309909: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

In [ ]:
model_accuracy(selected_features_20,X,X_test,y)

In [ ]:
model_accuracy(selected_features_25,X,X_test,y)

In [ ]:
model_accuracy(selected_features_30,X,X_test,y)

In [ ]:
model_accuracy(selected_features_35,X,X_test,y)

In [ ]:
model_accuracy(selected_features_40,X,X_test,y)